In [22]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Balance Dataset

In [42]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

### Standardize and PreProcess Further

In [43]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [50]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [51]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count ,np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)


1787.0 3579 0.4993014808605756
228.0 447 0.5100671140939598
222.0 448 0.4955357142857143


In [52]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets = test_targets)

### Machine Learning!

In [59]:
input_size = 10
output_size = 2
hidden_layer_size = 50  # Change this to improve convergence time

# Two hidden layers, classification so exit layer is softmax

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(output_size, activation='softmax'),    
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5)

model.fit(train_inputs,
          train_targets,
         batch_size = batch_size,
         epochs = max_epochs,
         callbacks=[early_stopping],
         validation_data = (validation_inputs, validation_targets),
         verbose = 2
         )

Epoch 1/100
112/112 - 1s - loss: 0.5852 - accuracy: 0.6935 - val_loss: 0.4457 - val_accuracy: 0.7673 - 678ms/epoch - 6ms/step
Epoch 2/100
112/112 - 0s - loss: 0.4015 - accuracy: 0.7871 - val_loss: 0.3793 - val_accuracy: 0.8009 - 347ms/epoch - 3ms/step
Epoch 3/100
112/112 - 0s - loss: 0.3653 - accuracy: 0.8013 - val_loss: 0.3683 - val_accuracy: 0.7919 - 359ms/epoch - 3ms/step
Epoch 4/100
112/112 - 0s - loss: 0.3553 - accuracy: 0.8058 - val_loss: 0.3639 - val_accuracy: 0.7852 - 380ms/epoch - 3ms/step
Epoch 5/100
112/112 - 0s - loss: 0.3475 - accuracy: 0.8083 - val_loss: 0.3671 - val_accuracy: 0.7875 - 380ms/epoch - 3ms/step
Epoch 6/100
112/112 - 0s - loss: 0.3450 - accuracy: 0.8075 - val_loss: 0.3656 - val_accuracy: 0.8009 - 356ms/epoch - 3ms/step
Epoch 7/100
112/112 - 0s - loss: 0.3392 - accuracy: 0.8159 - val_loss: 0.3644 - val_accuracy: 0.7897 - 349ms/epoch - 3ms/step
Epoch 8/100
112/112 - 0s - loss: 0.3329 - accuracy: 0.8206 - val_loss: 0.3564 - val_accuracy: 0.8121 - 351ms/epoch - 3

In [48]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3660 - accuracy: 0.7812
